<a href="https://colab.research.google.com/github/flari-gold/IU/blob/main/IU_RedditDataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Package Installs mit pip
!pip install praw
!pip install HanTa

#Import Modules
from google.colab import userdata
import praw
import collections
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from HanTa import HanoverTagger

In [ ]:
#Parameter für die Anmeldung
#Daten der unter https://old.reddit.com/prefs/apps/ generierten Anwendung. Ausreichend für ReadOnly Zugriffe.
#Die Daten sind im Google Colab Secrets hinterlegt
CLIENT_ID=userdata.get('CLIENT_ID')
CLIENT_SECRET=userdata.get('CLIENT_SECRET')

#Für Write Zugriffe muss ebenfalls noch der eigene Account angegeben werden, kann in diesem Fall ignoriert werden
#USERNAME=userdata.get('USERNAME')
#PASSWORD=userdata.get('PASSWORD')

#Parameter für die Datenanalyse
SUBREDDIT='linuxmemes'
ANZAHL_POSTS=100
TOPICS=5
PASSES=10
COMMENTLIMIT=None

In [ ]:
#Anfrage an die API und Speicherung

#Initialisierung reddit API
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent="Mein Topic Modeling Script", #User Agent ist zwingend notwendig
    check_for_async=False #PRAW wirft eine Warnung welche die ASYNC API empfiehlt, kann ignoriert werden
)

#Counter für die Metriken
user_upvotes = collections.Counter()
user_contributionscounter = collections.Counter()

all_textdata = []
#Anfragen an die Reddit API mit über request/http liefern die Daten im JSON Format. PRAW vereinfacht dies, in dem man die direkt die Attribute ansprechen kann
for submission in reddit.subreddit(SUBREDDIT).hot(limit=ANZAHL_POSTS):

    #Hochzählen der Counter
    if submission.author != None:
        user_contributionscounter[submission.author] += 1
        user_upvotes[submission.author] += submission.score

    post_textdata = [submission.title]#Alle Reddit Beiträge haben einen Titel. Der Titel wird ebenfalls im Topic Modeling wie ein normaler Kommentar behandelt
    post_textdata.append(submission.selftext)#Nicht alle Beträge haben einen Selftext. Beiträge die auf Links verweisen beispielsweise. Falls Selftext vorhanden ist, wird er im Topic Modeling behandelt, ansonsten ist der String leer

#Sobald ein Kommntar auf Reddit zu viele Antworten hat, werden die Antworten hinter einer Schaltfläche "more" verborgen und das gleiche gilt auch beim Zugriff auf die API.
#Das Auflösen der "more" stellt einen eigenen API Aufruf da und sorgt aufgrund der Zugriff Limits bei größeren Anfragen zu einer langen Laufzeit
    submission.comments.replace_more(limit=COMMENTLIMIT)

    for comment in submission.comments.list():

        #Hochzählen der Counter
        if comment.author != None:
            user_contributionscounter[comment.author] += 1
            user_upvotes[comment.author] += comment.score

        post_textdata.append(comment.body)

    all_textdata.append(post_textdata)

#print(user_contributionscounter)
#print(user_upvotes)



In [ ]:
#Text Bereinigung

#Stopwörter von NLTK
stop_words = stopwords.words('english')
stop_words.extend(['would', 'get'])

#Lemmatiserung mit HanoverTagger
tagger_en = HanoverTagger.HanoverTagger('morphmodel_en.pgz')

all_cleantext = []
for i in all_textdata:

    string_result = []
    for j in i:

        templist = []
        #sehr unschöne Lösung meiner Meinung nach, Tokenizer sinnvoll
        tempstring = j.replace(".", "").replace(",", "").replace("!", "").replace("?", "").replace("(", "").replace(")", "").replace("-", " ").replace(";", " ").replace(":", "").replace("'", "").replace("’", "").replace("@", "").replace("+", " ").replace("-", " ").replace("[", "").replace("]", "").replace("*", "").replace("–", "").replace("/", "") #’–
        #Umwandlung in Kleinbuchstaben
        for word in tempstring.lower().split():
            #Lemmatisierung
            templist.append(tagger_en.analyze(word)[0])
        #Filterung von Stopwörtern
        templist = [word for word in templist if word not in stop_words]
        tempstring = ' '.join(templist)

        string_result.append(tempstring)
    all_cleantext.append(string_result)
    #print(string_result)



In [ ]:
#Erstellung des LDA Models mit Hilfe von Gensim
from gensim import corpora
from gensim.models import LdaModel

#all_cleantext ist eine Liste, welche Listen beinhaltet, welche wiederum Strings beinhalten
#Jeder String repräsentiert einen Post/Kommentar
#Für die Verarbeitung muss aber in der Liste für jedes Wort ein eigener String stehen
templist = []
for i in all_cleantext:
    long_string = ' '.join(i)
    templist.append(long_string.split())

allwords = corpora.Dictionary(templist)
allposts = [allwords.doc2bow(text) for text in templist]

lda_model = LdaModel(allposts, num_topics=TOPICS, id2word=allwords, passes=PASSES)





In [ ]:
#Ausgabe der Ergebnisse

#Ausgabe der Keywords pro Topic
print("Analyse des Subreddits: " + SUBREDDIT)

print("\nKeywords pro ermitteltem Topic")
for t,x in lda_model.print_topics():
    print("Topic {}: {}".format(t, x))

#Ausgabe der aktivsten User
print("\nMeiste Anzahl Beiträge pro User")
for t,x in user_contributionscounter.most_common(10):
    print("{}: {}".format(t, x))

#Ausgabe der User mit den meisten Upvotes
print("\nMeiste Anzahl Upvotes pro User")
for t,x in user_upvotes.most_common(10):
    print("{}: {}".format(t, x))

Analyse des Subreddits: linuxmemes

Keywords pro ermitteltem Topic

Topic 0: 0.014*"use" + 0.011*"Linux" + 0.011*"arch" + 0.010*"like" + 0.009*"fedora" + 0.007*"dont" + 0.006*"nuclear" + 0.006*"make" + 0.006*"one" + 0.005*"say"

Topic 1: 0.011*"Linux" + 0.010*"use" + 0.008*"window" + 0.006*"make" + 0.006*"file" + 0.005*"want" + 0.005*"french" + 0.005*"like" + 0.004*"best" + 0.004*"good"

Topic 2: 0.021*"use" + 0.011*"arch" + 0.008*"like" + 0.008*"Linux" + 0.007*"dont" + 0.006*"one" + 0.006*"user" + 0.006*"im" + 0.005*"work" + 0.005*"make"

Topic 3: 0.006*"comment" + 0.006*"use" + 0.005*"cat" + 0.004*"Linux" + 0.004*"post" + 0.004*"Meme" + 0.003*"Sudo" + 0.003*"^^|" + 0.003*"like" + 0.003*"im"

Topic 4: 0.014*"use" + 0.011*"Linux" + 0.010*"Ubuntu" + 0.008*"like" + 0.007*"window" + 0.007*"snap" + 0.007*"good" + 0.006*"distro" + 0.006*"dont" + 0.005*"make"

Analyse des Subreddits: ProgrammerHumor

Keywords pro ermitteltem Topic

Topic 0: 0.014*"use" + 0.009*"like" + 0.008*"count" + 0.007*"language" + 0.007*"length" + 0.007*"userid" + 0.006*"dont" + 0.006*"array" + 0.006*"make" + 0.006*"size"

Topic 1: 0.016*"c" + 0.015*"use" + 0.014*"tailwin" + 0.009*"like" + 0.008*"style" + 0.008*"class" + 0.008*"component" + 0.006*"make" + 0.006*"good" + 0.006*"dont"

Topic 2: 0.010*"use" + 0.010*"work" + 0.010*"code" + 0.008*"like" + 0.007*"make" + 0.006*"dont" + 0.005*"time" + 0.005*"one" + 0.004*"ai" + 0.004*"good"

Topic 3: 0.018*"file" + 0.015*"git" + 0.010*"use" + 0.009*"change" + 0.009*"work" + 0.008*"backup" + 0.008*"like" + 0.007*"make" + 0.007*"dont" + 0.006*"discard"

Topic 4: 0.011*"use" + 0.011*"commit" + 0.009*"like" + 0.006*"dont" + 0.006*"one" + 0.006*"work" + 0.006*"code" + 0.006*"make" + 0.006*"time" + 0.006*"git"

Analyse des Subreddits: Slipknot

Keywords pro ermitteltem Topic

Topic 0: 0.012*"song" + 0.010*"like" + 0.010*"iowa" + 0.009*"album" + 0.007*"title" + 0.007*"love" + 0.007*"one" + 0.007*"slipknot" + 0.007*"self" + 0.005*"dont"

Topic 1: 0.012*"im" + 0.010*"like" + 0.008*"thats" + 0.008*"slipknot" + 0.007*"make" + 0.007*"people" + 0.007*"think" + 0.006*"one" + 0.006*"go" + 0.006*"corey"

Topic 2: 0.020*"album" + 0.017*"like" + 0.017*"song" + 0.011*"time" + 0.011*"first" + 0.010*"see" + 0.010*"dont" + 0.008*"one" + 0.008*"slipknot" + 0.008*"good"

Topic 3: 0.012*"slipknot" + 0.009*"one" + 0.009*"Sic" + 0.007*"song" + 0.007*"go" + 0.006*"cool" + 0.006*"make" + 0.006*"good" + 0.006*"dont" + 0.006*"like"

Topic 4: 0.009*"version" + 0.009*"iowa" + 0.009*"self" + 0.009*"title" + 0.008*"one" + 0.008*"corey" + 0.008*"mfkr" + 0.008*"album" + 0.008*"slipknot" + 0.007*"cover"

In [ ]:
#Nicht teil der Aufgabe, aber hilft zur Interpretation
#Paket zur Visualisierung
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim

In [ ]:
#Erstellt Grafik
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda_model, allposts, allwords)
pyLDAvis.display(panel)

In [ ]:
#Nicht teil der Aufgabe
#Auflistung der häufigsten Wörter
templist = []
for i in all_cleantext:
    long_string = ' '.join(i)
    templist.append(long_string)
end_string = ' '.join(templist)


print(collections.Counter(end_string.split()).most_common(10))